## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-_okjjfud because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


## Import data

### 1b. Filled ISH

In [2]:
ish = pd.read_csv("/beegfs/home/pmatyskova/project/ish_data_cor_hypnoSFO_filled1_0.csv") #filled with 0 only

In [3]:
ish_full_genenames = np.load("/beegfs/home/pmatyskova/project/ish_data_cor_genenames.npy")

ish = ish.drop(['Unnamed: 0'], axis=1)
ish.columns = ish_full_genenames
ish = np.transpose(ish)
ish['Unnamed: 0'] = ish.index

### 2. scRNAseq

In [4]:
hyp = pd.read_csv('/beegfs/scratch/bruening_scratch/lsteuernagel/data/petra_data/hypoMap_avg_signatures/hypomap_avg_signatures_C185_rna.txt', sep = "\t")
hyp.index = hyp.iloc[:,0]
hyp = hyp.drop(['Unnamed: 0'], axis=1)

genes_to_exclude = pd.read_csv('/beegfs/scratch/bruening_scratch/lsteuernagel/projects/analysis_projects/volumetric_analysis/genes_to_exclude.csv',
                               header = None)
hyp = hyp[~hyp.index.isin(genes_to_exclude.iloc[:,0])] #exclude the genes to exclude from hypomap output

### Random probe selection

In [ ]:
#remove duplicates: random pick
ish = ish.sample(frac=1, random_state=1) 
ish = ish.drop_duplicates(subset = ['Unnamed: 0'], keep='first')
ish = ish.drop(['Unnamed: 0'], axis=1)

In [6]:
ish = ish[ish.index.isin(hyp.index)] #keep the common ones
hyp = hyp[hyp.index.isin(ish.index)] #keep the common ones

In [7]:
match_index = [list(hyp.index).index(i) for i in list(ish.index)] #find indeces to reorder hyp to ish
hyp = hyp.iloc[match_index,] #reorder hyp to match ish

In [8]:
gene_annot = hyp.index #save gene names to know what genes selected
cellt_annot = hyp.columns #save cellt names to annotate np array later

c = hyp.to_numpy()
e = ish.to_numpy()

### Save

In [ ]:
np.save(file = "/beegfs/home/pmatyskova/project/ishfstep_c_hm185cor_randp_filled0", arr = c)
np.save(file = "/beegfs/home/pmatyskova/project/ishfstep_e_hm185cor_randp_filled0", arr = e)
np.save(file = "/beegfs/home/pmatyskova/project/ishfstep_geneset_hm185cor_randp_filled0", arr = gene_annot)